<h1>Read the linearly seperable classification dataset files and prepare the data for training, testing, and validation</h1>

In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

### Create data set by combining all three class data
def all_class_data(train1, train2, train3, validate1, validate2, validate3, test1, test2, test3):
    trainX = np.concatenate((train1, train2, train3), axis=0)
    trainY = np.array([0 for i in range(len(train1))] + [1 for i in range(len(train2))] + [2 for i in range(len(train3))])
    validateX = np.concatenate((validate1, validate2, validate3), axis=0)
    validateY = np.array([0 for i in range(len(validate1))] + [1 for i in range(len(validate2))] + [2 for i in range(len(validate3))])
    testX = np.concatenate((test1, test2, test3), axis=0)
    testY = np.array([0 for i in range(len(test1))] + [1 for i in range(len(test2))] + [2 for i in range(len(test3))])
    return trainX, trainY, validateX, validateY, testX, testY

#### Path of all class dataset 
f1 = r"D:\Sujeet_PhD\Course_Work\DeepLearning (CS671)\Assignment\Assignment1\dataset\Group32\Classification\LS_Group32\Class1.txt"
f2 = r"D:\Sujeet_PhD\Course_Work\DeepLearning (CS671)\Assignment\Assignment1\dataset\Group32\Classification\LS_Group32\Class2.txt"
f3 = r"D:\Sujeet_PhD\Course_Work\DeepLearning (CS671)\Assignment\Assignment1\dataset\Group32\Classification\LS_Group32\Class3.txt"

### Devide the Class1 data into training, validation, and testing data
df = pd.read_csv(f1, delimiter=' ', header=None)
train1, validate1, test1 = np.split(df, [int(0.6*len(df)), int(0.8*len(df))])
### Devide the Class2 data into training, validation, and testing data
df = pd.read_csv(f2, delimiter=' ', header=None)
train2, validate2, test2 = np.split(df, [int(0.6*len(df)), int(0.8*len(df))])
### Devide the Class3 data into training, validation, and testing data
df = pd.read_csv(f3, delimiter=' ', header=None)
train3, validate3, test3 = np.split(df, [int(0.6*len(df)), int(0.8*len(df))])

#### Combine all class dataset to prepare training, validation, and tesing dataset ####
trainX, trainY, validateX, validateY, testX, testY = all_class_data(train1, train2, train3, validate1, validate2, validate3, test1, test2, test3)

### Convert label data to one hot encoder
### 0 -> (1, 0, 0), 1 -> (0, 1, 0), 2 -> (0, 0, 1)
enc = OneHotEncoder()
y_OHE_train = enc.fit_transform(np.expand_dims(trainY,1)).toarray()
y_OHE_val = enc.fit_transform(np.expand_dims(validateY,1)).toarray()
y_OHE_test = enc.fit_transform(np.expand_dims(testY,1)).toarray()


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you us

<h1>Implementation of multilayer feed forward neural network (MLFFNN)</h1> 

In [15]:
import numpy as np
import math
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

### Activation Functions Definitions
class Sigmoid():
    def __call__(self, x, b=1):
        return 1.0/(1.0 + np.exp(-(b*x)))
    def gradient(self, x, b=1):
        return self.__call__(x, b) * (1 - self.__call__(x, b))

class Linear():
    def __call__(self, x, b=1):
        return b*x
    def gradient(self, x, b=1):
        return b

### Multilayer feed forward neural network class
class MLFFNN():
    def __init__(self, n_hidden, n_epoch=1000, learning_rate=0.01, threshold=0.001):
        self.n_hidden = n_hidden
        self.n_epoch = n_epoch
        self.learning_rate = learning_rate
        self.threshold = threshold
        self.hidden_activation = Sigmoid()
        self.output_activation = Sigmoid()

    ### Initialize the weights of neural network
    def initialize_weights(self, X, y):
        n_samples, n_features = X.shape
        _, n_outputs = y.shape
        
        ### For all hidden layers
        pre_num_of_neuron = n_features
        self.weights = {}
        self.w0 = {}
#         limit = 1
        for i in range(len(self.n_hidden)):
            limit   = 1 / math.sqrt(pre_num_of_neuron/2)
            self.weights[i]  = np.random.uniform(-limit, limit, (pre_num_of_neuron, self.n_hidden[i]))
            self.w0[i] = np.zeros((1, self.n_hidden[i]))
            pre_num_of_neuron = self.n_hidden[i]
        
        # For output layer
        limit   = 1 / math.sqrt(pre_num_of_neuron/2)
        self.V  = np.random.uniform(-limit, limit, (self.n_hidden[-1], n_outputs))
        self.v0 = np.zeros((1, n_outputs))

    def train(self, X, y, epoch=True):
        self.initialize_weights(X, y)
        self.errors = []
        ### This conditional block of code is for fixed number of epoch
        if epoch == True:
            ### Run it for n_epoch times
            for i in range(self.n_epoch):
                ### For hidden layer
                inputs = X
                self.hidden_input = {}
                self.hidden_output = {}
                ### Forward Calculation ###
                self.forward_calculation(inputs)
                ### Backpropagation Calculation ###
                self.backpropagation_calculation(inputs, y)

                ### Store average instantaneous errors for each epoch
                self.errors.append(np.sum(self.SquareLoss(y, self.y_pred))/y.shape[0])
        ### This conditional block of code is for fixed threshold of average error
        else:
            error = 10000000
            noOfNoChangeError = 0
            ### Run it until error converges to the threshhold
            while error > self.threshold:
                ### For hidden layer
                inputs = X
                self.hidden_input = {}
                self.hidden_output = {}
                ### Forward Calculation ###
                self.forward_calculation(inputs)
                ### Backpropagation Calculation ###
                self.backpropagation_calculation(inputs, y)
                
                ### Store average instantaneous errors for each epoch
                self.errors.append(np.sum(self.SquareLoss(y, self.y_pred))/y.shape[0])
                ### If there is no change in error
                if noOfNoChangeError >=20:
                    break
                else:
                    if len(self.errors) >= 4:
                        if error == self.errors[-1] and error == self.errors[-2]:
                            noOfNoChangeError += 1
                error = self.errors[-1]
                
    ### Forward Calculation ###
    def forward_calculation(self, inputs):
        ### For hidden layer
        for i in range(len(self.n_hidden)):
            ### Input to neuron
            self.hidden_input[i] = inputs.dot(self.weights[i]) + self.w0[i]
            ### Output of neuron
            self.hidden_output[i] = self.hidden_activation(self.hidden_input[i])
            inputs = self.hidden_output[i]
        ### For output layer
        self.output_layer_input = inputs.dot(self.V) + self.v0
        self.y_pred = self.output_activation(self.output_layer_input)
        return self.y_pred
    
    ### Backpropagation Calculation ###
    def backpropagation_calculation(self, inputs, y):
        ### First for output layer
        ### Gradient w.r.t input of output layer
        grad_wrt_out_l_input = self.loss(y, self.y_pred) * self.output_activation.gradient(self.output_layer_input)
        grad_v = self.hidden_output[len(self.n_hidden)-1].T.dot(grad_wrt_out_l_input)
        grad_v0 = np.sum(grad_wrt_out_l_input, axis=0, keepdims=True)
        ### For hidden layer
        ### Gradient w.r.t input of hidden layer
        next_grad_wrt_hidden_l_input = grad_wrt_out_l_input
        next_weight = self.V
        prev_input = inputs
        grad_w = {}
        grad_w0 = {}
        ### Calculation for multiple hidden layer starting from last to first hidden layer
        for i in reversed(range(len(self.n_hidden))):
            grad_wrt_hidden_l_input = next_grad_wrt_hidden_l_input.dot(next_weight.T) * self.hidden_activation.gradient(self.hidden_input[i])
            ### If hidden layer not connected to input layer
            if i != 0:
                grad_w[i] = self.hidden_output[i-1].T.dot(grad_wrt_hidden_l_input)
            ### when hidden layer connected to input layer
            else:
                grad_w[i] = inputs.T.dot(grad_wrt_hidden_l_input)
            grad_w0[i] = np.sum(grad_wrt_hidden_l_input, axis=0, keepdims=True)
            next_grad_wrt_hidden_l_input = grad_wrt_hidden_l_input
            next_weight = self.weights[i]

        ### Calculaton for weights update ###
        ### Weights update of output layer
        self.V  -= self.learning_rate * grad_v
        self.v0 -= self.learning_rate * grad_v0
        ### Weights update of hidden layers
        for i in range(len(self.n_hidden)):
            self.weights[i]  -= self.learning_rate * grad_w[i]
            self.w0[i] -= self.learning_rate * grad_w0[i]
    
    ### Prediction Function
    def predict(self, X):
        ### Call Forward Calculation ###
        y_pred = self.forward_calculation(X)
        return y_pred
    
    ### Instantaneous error and loss function
    def SquareLoss(self, y, y_pred):
        return 0.5 * np.power((y - y_pred), 2)
    def loss(self, y, y_pred):
        return -(y - y_pred)

### Calculate the accuracy
def accuracy_score(y, y_pred):
    accuracy = np.sum(y == y_pred, axis=0) / len(y)
    return accuracy*100

### To calculate the confusion matrix and classification accuracy
def confusion_matrix(actual, predicted):
    cm = np.zeros((3, 3))
    for i, j in zip(actual, predicted):
        cm[i][j] += 1
    ### For classification accuracy
    accuracy = np.sum(actual == predicted) * 100.0 / float(len(actual))
    return cm, accuracy

### Plot of Epoch vs Mean Square Error
def epochVsError_plot(model):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    error = model.errors
    nepoch = [i+1 for i in range(len(error))]
    plt.scatter(nepoch, error, marker='o', s=5, facecolors='b', edgecolors='b')
    plt.xlabel('Number of Epoch')
    plt.ylabel('Average Error')
#     plt.title('Average error vs number of epoch for training of Linearly Seperable data using MLFFNN')
    plt.savefig("AvgErrorVsEpoch_MLFFNN_LS.png", dpi=600, bbox_inches="tight")
    plt.clf()

### Decision Region Plot
def decision_boundary_plot(testX, model):
    ### Get the minimum and maximum limit for x-axis and y-axis from data
    x_min, x_max = testX[:, 0].min() - 1, testX[:, 0].max() + 1
    y_min, y_max = testX[:, 1].min() - 1, testX[:, 1].max() + 1
#     print(x_min, x_max, y_min, y_max)
    ### Create the data points from x-axis and y-axis values with some intervals
    h=0.05
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    g_data = np.c_[xx.ravel(), yy.ravel()]
    
    ### prediction of created data points
    predictedLabel = np.argmax(model.predict(g_data), axis=1)
    colors = ['#EE6363', '#BCEE68', '#B2DFEE']
    predictedColor = [colors[i] for i in predictedLabel]
    
    ### Plot input test data and decision region 
    plt.scatter(g_data[:,0], g_data[:,1], s=5, color=predictedColor)
    plt.scatter(np.array(test1)[:,0], np.array(test1)[:,1], s=5, color='red', label='Class1')
    plt.scatter(np.array(test2)[:,0], np.array(test2)[:,1], s=5, color='green', label='Class2')
    plt.scatter(np.array(test3)[:,0], np.array(test3)[:,1], s=5, color='blue', label='Class3')
    plt.legend(bbox_to_anchor=(0.05, 1.15), loc='upper left', ncol=3)
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.savefig("decision_boundry_MLFFNN_LS.png", dpi=600, bbox_inches="tight")
    plt.clf()

### 3D scatter plot for output of each nueron of hidden layer and output layer plot
def output_nueron_plot(X_train, y_train, z, label='hidden', layer='', nn=''):
    # Creating figure
    fig = plt.figure(figsize = (16, 9))
    ax = plt.axes(projection ="3d")
    # Add x, y gridlines 
    ax.grid(b = True, color ='grey', linestyle ='-.', linewidth = 0.3, alpha = 0.2) 
    # Creating color map
    color = ['red', 'green', 'blue']
    color_list = [color[i] for i in y_train]
    # Creating plot
    sctt = ax.scatter3D(X_train[:,0], X_train[:,1], z, color = color_list)
    ax.set_xlabel('X-axis', fontweight ='bold') 
    ax.set_ylabel('Y-axis', fontweight ='bold') 
    ax.set_zlabel('Z-axis (Neuron Output) ', fontweight ='bold')
    # save plot
    plt.savefig("Output_of_{}_layer_{}_Nueron_{}_LS.png".format(label, layer, nn), dpi=600, bbox_inches="tight")
    plt.clf()

def main():
    ### Call the MLFFNN calss 
    mlffnn = MLFFNN(n_hidden=[3,3], n_epoch=1000, learning_rate=0.01, threshold=0.001)
    ### Train the MLFFNN
    mlffnn.train(trainX, y_OHE_train, epoch=True)
#     mlffnn.train(trainX, y_OHE_train, epoch=False)
    
    ### Plots for output of each nueron of hidden layer and output layer plot
    for i in range(len(mlffnn.n_hidden)):
        for k in range(mlffnn.hidden_output[i].shape[1]):
            z = mlffnn.hidden_output[i][:,k]
            output_nueron_plot(trainX, trainY, z, label='hidden', layer=i+1, nn=k+1)
    for k in range(mlffnn.y_pred.shape[1]):
        z = mlffnn.y_pred[:,k]
        output_nueron_plot(trainX, trainY, z, label='output', nn=k+1)
            
    ### Prediction for validation data
    y_pred_val = np.argmax(mlffnn.predict(validateX), axis=1)
    y_val = np.argmax(y_OHE_val, axis=1)
    
    ### Prediction for test data
    y_pred_test = np.argmax(mlffnn.predict(testX), axis=1)
    y_test = np.argmax(y_OHE_test, axis=1)

    ### Calculate the accuracy and confusion matrix for validate and test data
    CM, Accuracy = confusion_matrix(y_val, y_pred_val)
    print("Confusion Matrix of Validate data: {}".format(CM))
    print("Classification Accuracy of Validate data: {}".format(Accuracy))
    ### Call the confusion_matrix function
    CM, Accuracy = confusion_matrix(y_test, y_pred_test)
    print("Confusion Matrix of Validate data: {}".format(CM))
    print("Classification Accuracy of Validate data: {}".format(Accuracy))
    
    ### Epoch vs error plot
    epochVsError_plot(mlffnn)
    
    ### Decision boundary plot
    decision_boundary_plot(testX, mlffnn)

if __name__ == "__main__":
    main()

Confusion Matrix of Validate data: [[100.   0.   0.]
 [  0. 100.   0.]
 [  0.   0. 100.]]
Classification Accuracy of Validate data: 100.0
Confusion Matrix of Validate data: [[100.   0.   0.]
 [  0. 100.   0.]
 [  0.   0. 100.]]
Classification Accuracy of Validate data: 100.0


<Figure size 1152x648 with 0 Axes>

<Figure size 1152x648 with 0 Axes>

<Figure size 1152x648 with 0 Axes>

<Figure size 1152x648 with 0 Axes>

...

<Figure size 1152x648 with 0 Axes>

<Figure size 1152x648 with 0 Axes>

<Figure size 1152x648 with 0 Axes>

<Figure size 1152x648 with 0 Axes>

<Figure size 432x288 with 0 Axes>